# Seasonality inspection

Explore liquidity, latency and spread metrics by hour-of-week.

In [ ]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import ipywidgets as widgets

data_path = pathlib.Path('data/seasonality_source/trades.parquet')
if data_path.exists():
    df = pd.read_parquet(data_path)
else:
    rng = pd.date_range('2024-01-01', periods=24*7, freq='H', tz='UTC')
    df = pd.DataFrame({
        'ts_ms': (rng.view('int64') // 10**6),
        'liquidity': np.random.lognormal(mean=0, sigma=1, size=len(rng)),
        'latency_ms': np.random.randint(50, 300, size=len(rng)),
        'spread_bps': np.random.uniform(1, 10, size=len(rng)),
    })

ts = pd.to_datetime(df['ts_ms'], unit='ms', utc=True)
df['hour_of_week'] = ts.dt.dayofweek * 24 + ts.dt.hour


In [ ]:
hour_slider = widgets.IntSlider(min=0, max=167, value=0, description='Hour')

@widgets.interact(hour=hour_slider)
def inspect(hour):
    subset = df[df['hour_of_week'] == hour]
    if subset.empty:
        print('No data for this hour')
        return
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    axes[0].hist(subset['liquidity'])
    axes[0].set_title('Liquidity')
    axes[1].hist(subset['latency_ms'])
    axes[1].set_title('Latency (ms)')
    axes[2].hist(subset['spread_bps'])
    axes[2].set_title('Spread (bps)')
    plt.show()
